# Grundlagen Neuronale Netze - einfache Operationen

https://bootcamp.codecentric.ai

In diesem Notebook wollen wir das einfache Beispiel aus dem Video nachvollziehen. 

Wir:
- definieren einen Input Tensor (aus einem 28x28 Pixel Bild von MNIST)
- wir normalisieren die Werte des Bildes
- wir definieren Matrizen mit Gewichten, die wir lernen wollen
- wir kombinieren Matrix Multiplikationen und Aktivierungsfunktionen, um aus einem Input mit 784 Pixeln einen Output mit 10 Werten zu erhalten
- wir definieren ein Label
- (wir optimieren die Gewichte, damit sie zum Label passen - kleiner Vorausblick auf kommende Videos)

Das folgende neuronale Netz ist keine besonders sinnvolle Architektur. Auch das Training mit nur einem Bild macht natürlich wenig Sinn. Es geht darum zu verstehen, welche Rechenoperationen "unter der Haube" eines neuronalen Netzes stattfinden. **Daher ist das ganze (hier) noch stark vereinfacht.**

Hier noch einmal das Bild, was wir versuchen in Code nachzuvollziehen:

![simple nn](simple_nn.png)

Beispiel mit PyTorch

Zunächst ein paar benötigte Imports:

In [ ]:
%matplotlib inline

import torch
import torchvision
import math

from torchvision import transforms
from matplotlib import pyplot

Folgender Tensor ist die interne Darstellung eines Bildes.

Es ist ein 28x28 Pixel Matrix mit Zahlenwerten von 0-255. 

(0 = schwarz, 255 = weiss, dazwischen Graustufen)

In [ ]:
img = torch.tensor(
       [[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,
          18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 170, 253,
         253, 253, 253, 253, 225, 172, 253, 242, 195,  64,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,  49, 238, 253, 253, 253, 253, 253,
         253, 253, 253, 251,  93,  82,  82,  56,  39,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,  18, 219, 253, 253, 253, 253, 253,
         198, 182, 247, 241,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,  80, 156, 107, 253, 253, 205,
          11,   0,  43, 154,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,  14,   1, 154, 253,  90,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 139, 253, 190,
           2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  11, 190, 253,
          70,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  35, 241,
         225, 160, 108,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  81,
         240, 253, 253, 119,  25,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          45, 186, 253, 253, 150,  27,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,  16,  93, 252, 253, 187,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0, 249, 253, 249,  64,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          46, 130, 183, 253, 253, 207,   2,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  39, 148,
         229, 253, 253, 253, 250, 182,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  24, 114, 221, 253,
         253, 253, 253, 201,  78,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,  23,  66, 213, 253, 253, 253,
         253, 198,  81,   2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,  18, 171, 219, 253, 253, 253, 253, 195,
          80,   9,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,  55, 172, 226, 253, 253, 253, 253, 244, 133,  11,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0, 136, 253, 253, 253, 212, 135, 132,  16,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]],
       dtype=torch.float)

So kann man sich die Dimensionen des Tensors anschauen:

In [ ]:
img.size()

Und so sieht es aus, wenn man die Zahlen als Bild interpretiert (eine Zahl 5 aus dem MNIST Datensatz):

In [ ]:
pyplot.imshow(img, cmap="gray")

Jetzt klopfen wir den Tensor flach. Aus einer 28 x 28 Matrix wird ein Vektor mit 784 "Input Pixeln". Das sind die gleichen Zahlenwerte - nur nicht mehr in 28 Reihen sondern alle in einer Reihe aneinander gehängt.

In [ ]:
input_tensor = img.flatten()

In [ ]:
input_tensor.size()

Jetzt schauen wir uns mal den Wert an Stelle 180 an:

In [ ]:
input_tensor[180]

Der Wert beträgt 170 ...

... nun schauen wir an was der größte Wert in dem Vektor ist:

In [ ]:
max(input_tensor)

Wie zu erwarten war, ist es 255 (weiß - sicher sind einige Pixel in dem Bild weiß - größere Zahlen kann es bei einem solchen Bild nicht geben).

Jetzt machen wir eine einfache "Normalisierung" und teilen alle Werte des Vektors durch 255.

Damit ändern wir den Zahlenbereich im Vektor von 0-255 auf 0-1. Mit diesem Schritt kann man Probleme beim Training verringern - vor allem bei tieferen neuronalen Netzen wird das sehr wichtig.

In [ ]:
normalized_input_tensor = input_tensor / 255

Der Wert an der Stelle 180 (den wir vorher schon angesehen haben) ist jetzt 0.6667

Die Zahlen stehen aber noch im gleichen Verhältnis 0,66 ist 2/3 von 1 sowie 170 2/3 von 255 ist.

In [ ]:
normalized_input_tensor[180]

In [ ]:
max(normalized_input_tensor)

Wie zu erwarten ist die größte Zahl im Vektor jetzt 1

Jetzt initialisieren wir unsere erste Weight Matrix mit Parametern, die gelernt werden können. Anders als im Video wählen wir nicht 784x3 sondern 784x20 - im Video wurde nur eine kleinere Zahl gewählt, damit es auf eine Folie passt und übersichtlicher aussieht. 

Die Zahlen sind zunächst (kleine) Zufallszahlen. (was requires_grad bedeutet überspringen wir an dieser Stelle - dazu kommen wir später)

Wir teilen die Zufallszahlen durch die Wurzel aus der Größe der Input Schicht - wir tun dies, um die Weights auf einen "sinnvollen" Werte-Bereich zu initialisieren. Wenn wir das nicht tun, kann es sein, dass die Weights zu groß sind und unser neuronales Netz nicht lernt. Wenn man später als "Practitioner" libraries wie fast.ai verwendet muss man sich i.d.R. um solche Details nicht mehr sorgen - man sollte jedoch mal gesehen haben welche kleinen Änderungen wichtig sind und welche Auswirkungen sie haben.

In [ ]:
weights_tensor = torch.randn((784, 20)) / math.sqrt(784)
weights_tensor.requires_grad_()

In [ ]:
weights_tensor.size()

Jetzt berechnen wir wie im Video zuvor einige "Activations". Dazu machen wir eine Matrix-Multiplikation mit dem Input @ weights

In [ ]:
first_activation = normalized_input_tensor @ weights_tensor

In [ ]:
first_activation

Das ist das Ergebnis unserer ersten Matrix-Multiplikation.

Anders als im Video Beispiel hat diese jetzt auch wieder eine Size von 20, da wir ja eine größere Weight Matrix gewählt haben.

In [ ]:
first_activation.size()

Jetzt kommt die Aktivierungs-Funktion, um auch nicht lineare Zusammenhänge lernen zu können. 
Im Prinzip setzt diese alle negativen Activations aus dem vorigen Schritt auf 0.

In [ ]:
second_activation = first_activation.relu()

In [ ]:
second_activation

Jetzt initialisieren wir die zweite Weight Matrix (wie im Video Beispiel). Hier müssen wir jetzt auch wieder die Size von 3 auf 20 anpassen, damit die Matrix-Multiplikationen zusammen passen. Nach wie vor wollen wir aber eine Output Größe von 10 haben (in unserem Beispiel wollen wir ja Zahlen von 0-9) vorhersagen.

In [ ]:
more_weights_tensor = torch.randn((20, 10)) / math.sqrt(20)
more_weights_tensor.requires_grad_()

... eine weitere Matrix-Multiplikation ...

In [ ]:
output = second_activation @ more_weights_tensor

In [ ]:
output

In [ ]:
output.size()

Und das ist jetzt erstmal unser Output. Der Vektor hat die richtige Dimension. Die Zahlen darin sind bis hierhin erstmal **völlig bedeutungslos**.

Wir haben einmal das Modell mit Zufallszahlen durchgerechnet und geschaut, was am Ende raus kommt -> Zufallszahlen.

### Label definieren

An dieser Stelle definieren wir jetzt ein Label. Das Label ist das was wir vom neuronalen Netz erwarten. 

Wenn ich vorne ein Bild einer 5 rein gebe, dann soll folgendes herauskommen:

In [ ]:
label = torch.tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0], dtype=torch.float)

Diese verbreitete Form eines Labels nennt man auch "one hot encoded vector". Es ist Vektor der verschiedene Klassen abbilden kann - der Vektor an der Stelle 0 ist die Wahrscheinlichkeit für eine 0. Der Vector an der Stelle 5 ist die Wahrscheinlichkeit für eine 5 etc. (Es könnte aber auch etwas völlig anderes bedeuten - z.B. Stelle 0 = Katze, Stelle 3 = Hund etc.)

Da wir ja ein Bild einer 5 betrachten, soll also das label[5] = 1 und alles andere 0 sein.

Jetzt fügen wir dem Output noch eine weitere Aktivierungs-Funktion hinzu und haben unsere "prediction" - also unsere Vorhersage. Diese bringt die Zahlen Werte in einen Bereich von 0-1 (warum ist an dieser Stelle nicht relevant - wir tun es einfach :) )

In [ ]:
prediction = output.sigmoid()

Hier jetzt die aktuelle Vorhersage des Modells (gerundet, dass man es besser lesen kann):

In [ ]:
prediction.round()

### Loss Funktion

Unsere Loss Funktion soll uns den Fehler zwischen unserer prediction und dem label berechnen. Wir verwenden eine bestehende pytorch Funktion. (Warum genau diese, ist an dieser Stelle auch noch nicht relevant).

In [ ]:
loss_func = torch.nn.functional.binary_cross_entropy

Nun berechnen wir einmal beispielhaft den aktuellen Loss, also den Fehler oder den "Abstand" zwischen unserer prediction und dem label. 

In [ ]:
loss_func(prediction, label)

Das ist unser loss - was sagt uns das? Erstmal noch gar nichts (auch hier sind wir immer bei völlig aussagslosen Zufallszahlen.)

Jetzt machen wir eine "manuelle Vorhersage". Wir definieren einfach eine prediction wie sie uns gefällt. Sind bei dieser prediction mehr Einsen und Nullen an der richtigen Stelle sollte der folgende Loss kleiner weren - ansonsten größer:

In [ ]:
manual_prediction = torch.tensor([0, 0, 0, 0, 0, 1, 0, 1, 1, 0], dtype=torch.float)

In [ ]:
loss_func(manual_prediction, label)

Was passiert wenn alle Zahlen richtig vorhergesagt werden? Wenn unsere prediction gleich dem label ist?

In [ ]:
loss_func(label, label)

... der Loss geht gegen 0.

# Modell optimieren

(Kleiner Vorausblick)

Jetzt optimieren wir in ein paar Schritten unsere weight so, dass die prediction möglichst nah an das label heran kommt. Das ist das (vereinfachte) Prinzip, wie neuronale Netze lernen. Wir werden es in einem folgenden Video noch genauer betrachten.

Daher gehe ich einfach die Schritte durch, ohne diese detailliert zu erkläeren.

Zunächst berechnen wir den Loss als Tensor.

In [ ]:
loss = loss_func(prediction, label)
loss

Wir fordern pytorch dazu auf eine "Backpropagation" zu machen und die Gradienten für die weight Matrizen zu ermitteln.

In [ ]:
loss.backward()

In [ ]:
more_weights_tensor.grad.data

### Model definieren

Hier definieren wir einfach nochmal die gleichen Berechnungen wir zuvor - nur in einer Funktion, so dass wir sie in einer Schleife immer wieder aufrufen können.

In [ ]:
def model(x):
    return ( ((x @ weights_tensor).relu()) @ more_weights_tensor).sigmoid()

In [ ]:
model(normalized_input_tensor)

Wir machen eine prediction mit dem aktuellen Modell für unseren input Tensor (das Bild der 5):

In [ ]:
new_pred = model(normalized_input_tensor)
new_pred

... ermitteln den loss (als den Fehler zwischen Vorhersage und label) und machen eine backpropagation

In [ ]:
loss = loss_func(new_pred, label)
loss.backward()

Jetzt verwenden wir die Gradienten, um die Weights ein kleines bisschen in die richtige Richtung zu optimieren:

In [ ]:
lr = 0.1
with torch.no_grad():
    weights_tensor -= weights_tensor.grad * lr
    more_weights_tensor -= more_weights_tensor.grad * lr
    weights_tensor.grad.zero_()
    more_weights_tensor.grad.zero_()

In [ ]:
new_pred = model(normalized_input_tensor)
loss_func(new_pred, label)

... und wir sehen, dass unser Fehler tatsächlich etwas kleiner geworden ist.

## iterative Optimierung

Wenn wir diese einfachen Optimierungsschritte jetzt ganz oft aufrufen, dann werden die Gewichte immer mehr so angepasst, dass sich die prediction immer mehr dem label annähert (der loss kleiner wird):

In [ ]:
lr = 0.1
for i in range(1000):
    new_pred = model(normalized_input_tensor)
    loss = loss_func(new_pred, label)
    loss.backward()

    with torch.no_grad():
        weights_tensor -= weights_tensor.grad * lr
        more_weights_tensor -= more_weights_tensor.grad * lr
        weights_tensor.grad.zero_()
        more_weights_tensor.grad.zero_()

    if (i % 100 == 0): print("Loss: ", loss_func(new_pred, label).item())

Nach der Optimierung sieht unser Vorhersage nun so aus: 

In [ ]:
model(normalized_input_tensor)

Mit dieser Schreibweise ist auf einen ersten Blick erstmal nicht viel anzufangen. Schauen wir uns an welche Zahl am größten ist:

In [ ]:
model(normalized_input_tensor).argmax()

Die Zahl an der Stelle 5 ist die Größte. Also die 5, die wir vorhersagen wollen.

Runden wir die Zahlen auf und ab:

In [ ]:
model(normalized_input_tensor).round()

...sieht man deutlich besser, dass wir die Gewichte so optimiert haben, dass die prediction für die Pixel-Werte einer 5 zu dem gleichen Output Vector führen wir unser Label, das wir definiert haben.

# Fazit

Was wir hier gemacht haben, war ein vereinfachtes Beispiel dafür, welche Rechenoperationen in einem neuronalen Netz stattfinden. Wir haben nur mit einem Bild "trainiert" und die Gewichte auf dieses Bild "overfittet". Auch die Architektur dieses Netzes ist nicht unbedingt "eine Standard-Architektur" (- man muss solche Architekturen auch nicht unbedingt selbst erfinden können).  

ABER: Das was wir hier gesehen haben sind die Building Blocks von neuronalen Netzen. Das passiert unter der Haube. Matrix-Multiplikationen, Aktivierungsfunktionen, Ermittlung des Loss/Fehlers und Optimierungs-Schritte, um die Weights anzupassen - mehr nicht. Für moderne Deep Learning Verfahren und "richtiges Training" mit großen Datenmengen sind noch ein paar mehr Dinge nötig. Wir wollen ja nicht eine 5 "auswändig lernen" sondern generalisieren - trotzdem behalte dieses einfache Beispiel im Hinterkopf.


# Aufgaben

1. Verändere die Dimensionen der ersten weight Matrix auf 784x3 wie ursprünglich im Video Beispiel und versuche erneut das neuronale Netz zu optimieren.
2. Wie groß ist nun der Tensor mit den Aktivierungen nach der relu Funktion?
3. Was kann an dieser Stelle mit hoher Wahrscheinlichkeit passieren? Warum führt das dazu, dass das neuronale Netz nicht optimiert werden kann?
4. Ersetze bei der Initilisierung der weight-Matrizen das `/ math.sqrt(n)` durch `* 10`. Versuche erneut das neuronale Netz zu optimieren. Was passiert?

### Credits

Das Notebook ist inspiriert von folgendem Artikel von Jeremy Howard: https://pytorch.org/tutorials/beginner/nn_tutorial.html

